In [1]:
import pandas as pd
import numpy as np
import cv2
import math
import matplotlib.pyplot as plt

import PIL.Image as Image
import pathlib
import os

import tensorflow as tf
import tensorflow_hub as hub

from tensorflow import keras
from keras.preprocessing import image
from tensorflow.keras.utils import img_to_array
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [2]:
data_dir = pathlib.Path('../input/nigerianfooddataset/NigerianFood 3')
image_paths = list(data_dir.glob(r'**/*.jpg'))

In [3]:
len(image_paths)

In [4]:
# Make a copy of the image paths list
image_paths_copy = image_paths.copy()

In [5]:
# Create shuffle the list so i will be able to split list into 3 lists with random values
import random

random.shuffle(image_paths_copy)

In [6]:
# Split list into 3
from itertools import islice

length_to_split = [440, 100, 100]


input = iter(image_paths_copy)
output = [list(islice(input, elem))
         for elem in length_to_split]

In [7]:
# Create train, validation and test data
train_images_path = output[0]
valid_images_path = output[1]
test_images_path = output[2]

In [8]:
food_labels = []

for filename in os.listdir(data_dir):
    labels = filename.split('/')[-1]
    food_labels.append(labels)
    
food_labels

In [9]:
def image_processing(filepath):
    labels = [str(filepath[i]).split('/')[-2]
             for i in range(len(filepath))]
    
    filepath = pd.Series(filepath, name = 'Filepath').astype(str)
    labels = pd.Series(labels, name = 'Label')
    
    df = pd.concat([filepath, labels], axis='columns')
    
    return df

In [10]:
train_df = image_processing(train_images_path)
test_df = image_processing(test_images_path)
val_df = image_processing(valid_images_path)

In [11]:
# Create DataFrame with just one label for each food
df_unique = train_df.copy().drop_duplicates(subset=['Label']).reset_index()

In [12]:
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
preprocessing_function = tf.keras.applications.mobilenet_v2.preprocess_input
)

test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
preprocessing_function = tf.keras.applications.mobilenet_v2.preprocess_input
)

In [13]:
train_images = train_generator.flow_from_dataframe(
    dataframe  = train_df,
    x_col = 'Filepath',
    y_col = 'Label',
    target_size = (224, 224),
    color_mode = 'rgb',
    class_mode = 'categorical',
    batch_size = 32,
    shuffle = True,
    seed = 0,
    rotation_range = 30,
    zoom_range = 0.15,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.15,
    horizontal_flip = True,
    fill_mode = 'nearest'
)

In [14]:
val_images = train_generator.flow_from_dataframe(
    dataframe  = val_df,
    x_col = 'Filepath',
    y_col = 'Label',
    target_size = (224, 224),
    color_mode = 'rgb',
    class_mode = 'categorical',
    batch_size = 32,
    shuffle = True,
    seed = 0,
    rotation_range = 30,
    zoom_range = 0.15,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.15,
    horizontal_flip = True,
    fill_mode = 'nearest'
)

In [15]:
test_images = test_generator.flow_from_dataframe(
    dataframe  = test_df,
    x_col = 'Filepath',
    y_col = 'Label',
    target_size = (224, 224),
    color_mode = 'rgb',
    class_mode = 'categorical',
    batch_size = 32,
    shuffle = False
)

In [16]:
pretrained_model = tf.keras.applications.MobileNetV2(
input_shape= (224, 224, 3),
include_top = False,
weights = 'imagenet',
pooling = 'avg'
)
pretrained_model.trainable = False

In [17]:
inputs = pretrained_model.input

x = tf.keras.layers.Dense(250, activation = 'relu')(pretrained_model.output)
x1 = tf.keras.layers.Dense(200, activation = 'relu')(x)
x2 = tf.keras.layers.Dense(150, activation = 'relu')(x1)
x3 = tf.keras.layers.Dense(100, activation = 'relu')(x2)
x4= tf.keras.layers.Dense(50, activation = 'relu')(x3)


outputs = tf.keras.layers.Dense(8, activation = 'softmax')(x)

model = tf.keras.Model(inputs = inputs, outputs = outputs)

model.compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

In [18]:
history = model.fit(
    train_images,
    validation_data = val_images,
    batch_size = 32,
    epochs = 10,
    callbacks = [
        tf.keras.callbacks.EarlyStopping(
            monitor = 'val_loss',
            patience = 2,
            restore_best_weights = True
        )
        
    ]
)

In [19]:
# Predict the label of the test images
pred = model.predict(test_images)
pred = np.argmax(pred, axis=1)

# Map the label
labels = (train_images.class_indices)
labels = dict((v,k) for k, v in labels.items())
pred1 = [labels[k] for k in pred]

In [20]:
def output(imagepath):
    img = image.load_img(imagepath, target_size=(224, 224, 3))
    img = img_to_array(img)
    img = img/255
    img = np.expand_dims(img, [0])
    answer = model.predict(img)
    y_class = answer.argmax(axis=-1)
    y = " ".join(str(x) for x in y_class)
    y = int(y)
    res = labels[y]
    return res

In [21]:
img = output('../input/nigerianfooddataset/NigerianFood 3/Egusi/11.EGUSI-SOUP.jpg')
img

In [22]:
model.save('NF.h5')